In [1]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from generater import ChangableBC, BC, Domain

import pickle

In [2]:
class mynet(nn.Module):
    def __init__(self):
        super(mynet,self).__init__()

        self.main_node = [2,5,7,5,3,1]

        sum_of_node = 0
        for i in self.main_node:
            sum_of_node += i

        self.BCNet = nn.Sequential(
            nn.Linear(1200,600),
            nn.Tanh(),
            nn.Linear(600,300),
            nn.Tanh(),
            nn.Linear(300,100),
            nn.Tanh(),
            nn.Linear(100,50),
            nn.Tanh(),
            nn.Linear(50,sum_of_node - 1)
        )

        layer_list = []

        for i,j in zip(self.main_node[:-1],self.main_node[1:]):
            layer_list.append(nn.Linear(i*2,j))

        self.mainNet = nn.ModuleList(layer_list)
    
    def forward(self,x):
        x_for_main = x[:,:2]
        x_for_BC = x[:,2:]

        additional_input = self.BCNet(x_for_BC)


        input = x_for_main
        start = 0
        end = self.main_node[0]

        for i,layer in enumerate(self.mainNet):
            input = torch.hstack((input,additional_input[:,start:end]))
            input = layer(input)

            start += self.main_node[i]
            end += self.main_node[i+1]

        return input

In [3]:
model = mynet()
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [17]:
num_train = 10000
num_test = 100

train_dataset = ChangableBC(Domain([0,0.01],[0,0.01]),
                            n_data = num_train,
                            num_each_bc = 100,
                            max_noize = 20).set_BC_value(up=[500,'value'],
                                                         down=[300,'value'],
                                                         right=[400,'value'],
                                                         left=[300,'value'])
test_dataset = BC(Domain([0,0.01],[0,0.01]),
                    n_data = num_test,
                    num_each_bc = 100).set_BC_value(up=[490,'value'],
                                                    down=[315,'value'],
                                                    right=[360,'value'],
                                                    left=[270,'value'])
test_dataloader = DataLoader(test_dataset, batch_size = 8 , shuffle=True)

In [11]:
def equation(net,input):
    x = input[:,0].view(-1,1)
    y = input[:,1].view(-1,1)
    space = torch.hstack([x,y])
    output = net(torch.hstack((space,input[:,2:])))
    dTdx = torch.autograd.grad(output.sum(),x,create_graph=True)[0]
    dTdy = torch.autograd.grad(output.sum(),y,create_graph=True)[0]
    ddTddx = torch.autograd.grad(dTdx.sum(),x,retain_graph=True)[0]
    ddTddy = torch.autograd.grad(dTdy.sum(),y,retain_graph=True)[0]

    return ddTddx + ddTddy

In [19]:
import time

In [23]:
num_epoch = 100

for epoch in range(1,num_epoch+1):
    Total_Loss = 0
    Total_data_size = 0

    train_dataset.changeBC()
    train_dataloader = DataLoader(train_dataset, batch_size = 128, shuffle = True)
    for domain_data, labels in train_dataloader:
        init_s = time.perf_counter()
        
        optimizer.zero_grad()

        init_e = time.perf_counter()
        
        BC_data, interior_data = domain_data
        Residual_label, BC_label = labels

        data_e = time.perf_counter()
        
        BC_pred = model(BC_data)
        BC_loss = loss_fn(BC_pred, BC_label)

        BC_e = time.perf_counter()

        Residual_pred = equation(model,interior_data)

        Residual = loss_fn(Residual_pred, Residual_label)

        R_e = time.perf_counter()

        total_loss = BC_loss + Residual
        
        Total_Loss += total_loss * len(BC_data)
        Total_data_size += len(BC_data)

        Loss_e = time.perf_counter()

        total_loss.backward()

        back_e = time.perf_counter()

        optimizer.step()

        step_e = time.perf_counter()

        print("zero grad : {:.6f} %\ndata load : {:.6f} %\nBC        : {:.6f} %\nResidual  : {:.6f} %\nLoss      : {:.6f} %\nbackward  : {:.6f} %\nstep      : {:.6f} %".format((init_e - init_s) / (step_e - init_e) * 100, (data_e - init_e) /(step_e - init_e) * 100, (BC_e - data_e)/(step_e - init_e) * 100, (R_e - BC_e)/(step_e - init_e) * 100, (Loss_e - R_e)/(step_e - init_e) * 100, (back_e - Loss_e)/(step_e - init_e) *100, (step_e - back_e)/(step_e - init_e)*100))

        print("iteration : {:.4f} %".format(Total_data_size / num_train * 100),end = '\r')

    Total_Loss /= Total_data_size
    print(f"Epoch : [{epoch}/{num_epoch}], Loss : {Total_Loss:.4f}")

zero grad : 0.015271 %
data load : 0.157541 %
BC        : 3.585323 %
Residual  : 26.311691 %
Loss      : 0.001990 %
backward  : 69.776838 %
step      : 0.166616 %
zero grad : 0.007682 %
data load : 0.003640 %
BC        : 3.637081 %
Residual  : 27.382451 %
Loss      : 0.001808 %
backward  : 68.785169 %
step      : 0.189852 %
zero grad : 0.013174 %
data load : 0.003874 %
BC        : 3.660656 %
Residual  : 26.826645 %
Loss      : 0.001883 %
backward  : 69.375256 %
step      : 0.131687 %
zero grad : 0.007578 %
data load : 0.008294 %
BC        : 3.655596 %
Residual  : 27.761902 %
Loss      : 0.001921 %
backward  : 68.390759 %
step      : 0.181527 %
zero grad : 0.007405 %
data load : 0.003733 %
BC        : 3.626552 %
Residual  : 26.292461 %
Loss      : 0.002387 %
backward  : 69.921400 %
step      : 0.153467 %
zero grad : 0.013003 %
data load : 0.007123 %
BC        : 3.595852 %
Residual  : 27.658383 %
Loss      : 0.002110 %
backward  : 68.577926 %
step      : 0.158605 %
zero grad : 0.008207 %

KeyboardInterrupt: 